#### Name: HARIPRASAD SIVAPATHAM ANAND
#### Student Number: 202205341
#### Email: x2022gmv@gmail.com
#### Accuracy Achieved: 77.0333%

In [339]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle

## Dataset Description

Variable Definition Key

survival: Survival 0 = No, 1 = Yes

pclass Ticket class 1 = 1st, 2 = 2nd, 3 = 3rd

sex Male or Female

Age Age in years

sibsp # of siblings / spouses aboard the Titanic

parch # of parents / children aboard the Titanic

ticket Ticket number

fare Passenger fare

cabin Cabin number

embarked Port of Embarkation C = Cherbourg, Q = Queenstown, S = Southampton

Variable Notes pclass: A proxy for socio-economic status (SES) 1st = Upper 2nd = Middle 3rd = Lower

age: Age is fractional if less than 1. If the age is estimated, is it in the form of xx.5

sibsp: The dataset defines family relations in this way...

Sibling = brother, sister, stepbrother, stepsister

Spouse = husband, wife (mistresses and fiancés were ignored)

parch: The dataset defines family relations in this way...

Parent = mother, father

Child = daughter, son, stepdaughter, stepson

Some children traveled only with a nanny, therefore parch=0 for them

In [340]:
dataFrame = pd.read_csv("./train.csv")

In [341]:
def get_data_info(dataFrame):
    print(dataFrame.info())

In [342]:
get_data_info(dataFrame)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB
None


In [343]:
# Check Values in non unique columns
for i in dataFrame:
    if(dataFrame[i].is_unique == False and len(dataFrame[i].unique())<10):
        print(i,dataFrame[i].unique())

Survived [0 1]
Pclass [3 1 2]
Sex ['male' 'female']
SibSp [1 0 3 4 2 5 8]
Parch [0 1 2 5 3 4 6]
Embarked ['S' 'C' 'Q' nan]


# Understanding the data 

In the following dataset the following columns are unique
- PassenderId, Name

The following are categorical variables with less than 10 features
- Survived, Pclass, Sex, SibSp, Parch, Embarked

The following are numerical 
- Age, Fare

**The Cabin**
- The cabin column has more than 10 categories, but it has more than 50% data missing hence we can drop the column

**Survived**
- We don't have to any changes to survived as survived is already in binary format

In [344]:
# Dropping Cabin Column
dataFrame = dataFrame.drop("Cabin",axis=1)

In [345]:
dataFrame.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(4)
memory usage: 76.7+ KB


## Handel null values

In [346]:
# As we need all the data to extract maximum information we wil do the following 

# Replace age with mean as age is a numerical data type
age_mean = dataFrame["Age"].mean()
dataFrame.Age =  dataFrame.Age.fillna(age_mean)
# Replace Embarked with mode or maximum value 
count_of_embarked = dataFrame.Embarked.value_counts().idxmax()
dataFrame.Embarked = dataFrame.Embarked.fillna(count_of_embarked)
# We can remove passengerId,name as all the values are unique 
dataFrame = dataFrame.drop(["PassengerId","Name"],axis=1)
# We can also drop Ticket as Tickets as most of the values as 681 values are unique and encoding will increase the dimensions
dataFrame = dataFrame.drop(["Ticket"],axis=1)

In [347]:
get_data_info(dataFrame)
# All the Missing values have been handel

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Survived  891 non-null    int64  
 1   Pclass    891 non-null    int64  
 2   Sex       891 non-null    object 
 3   Age       891 non-null    float64
 4   SibSp     891 non-null    int64  
 5   Parch     891 non-null    int64  
 6   Fare      891 non-null    float64
 7   Embarked  891 non-null    object 
dtypes: float64(2), int64(4), object(2)
memory usage: 55.8+ KB
None


# Feature Engineering

In [348]:
# https://triangleinequality.wordpress.com/2013/09/08/basic-feature-engineering-with-the-titanic-data/

# As Parch and SibSp basically means number of parents and number of sibillings we can combine both features and make it into a single column
def combine_parch_sibsp(dataFrame,parch,sibsp):
    new_column_name = "total_members"
    dataFrame["total_members"] = parch+sibsp
    return dataFrame;

In [349]:
dataFrame = combine_parch_sibsp(dataFrame,dataFrame.Parch,dataFrame.SibSp)

In [350]:
get_data_info(dataFrame)
# We have reduced the number of dimensions from 10 to 8

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Survived       891 non-null    int64  
 1   Pclass         891 non-null    int64  
 2   Sex            891 non-null    object 
 3   Age            891 non-null    float64
 4   SibSp          891 non-null    int64  
 5   Parch          891 non-null    int64  
 6   Fare           891 non-null    float64
 7   Embarked       891 non-null    object 
 8   total_members  891 non-null    int64  
dtypes: float64(2), int64(5), object(2)
memory usage: 62.8+ KB
None


In [351]:
# Using Feature Engineering we will convert Age and Fare to Catagorical Variables

In [352]:
dataFrame.describe()

,Survived,Pclass,Age,SibSp,Parch,Fare,total_members
count,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000
mean,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208,0.904602
std,0.486592,0.836071,13.002015,1.102743,0.806057,49.693429,1.613459
min,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,2.000000,22.000000,0.000000,0.000000,7.910400,0.000000
50%,0.000000,3.000000,29.699118,0.000000,0.000000,14.454200,0.000000
75%,1.000000,3.000000,35.000000,1.000000,0.000000,31.000000,1.000000
max,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200,10.000000


In [353]:
# https://towardsdatascience.com/from-numerical-to-categorical-3252cf805ea2

# Converting Numarical to catagoical objects

# Based on Calculation 
    # Age can be divided into 6 catagories 
    # Fare can be into 10 catagories
    # The following calculations were performed using 
        # Standard Deviation 
        # Maximum of column 
        # Minimum of column 

In [354]:
# https://pandas.pydata.org/docs/reference/api/pandas.cut.html

def age_to_category(dataFrame,column):
    bin_edge = [0,13,26,39,52,65,80];
    bin_labels = ["Young","Student","Teenage","Adult","Senior","Old"]
    dataFrame["AGE"] = pd.cut(column,bin_edge,labels=bin_labels)
    dataFrame = dataFrame.drop("Age",axis=1)
    return dataFrame;


In [355]:
dataFrame = age_to_category(dataFrame,dataFrame.Age)

In [356]:
# https://pandas.pydata.org/docs/reference/api/pandas.cut.html
def fare_to_category(dataFrame,column):
    bin_edge = [0,50,100,150,200,250,300,350,400,450,520];
    bin_labels = ["F1","F2","F3","F4","F5","F6","F7","F8","F9","F10"]
    dataFrame["FARE"] = pd.cut(column,bin_edge,labels=bin_labels);
    dataFrame = dataFrame.drop("Fare",axis=1)
    return dataFrame

In [357]:
dataFrame = fare_to_category(dataFrame,dataFrame.Fare)

In [358]:
dataFrame.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype   
---  ------         --------------  -----   
 0   Survived       891 non-null    int64   
 1   Pclass         891 non-null    int64   
 2   Sex            891 non-null    object  
 3   SibSp          891 non-null    int64   
 4   Parch          891 non-null    int64   
 5   Embarked       891 non-null    object  
 6   total_members  891 non-null    int64   
 7   AGE            891 non-null    category
 8   FARE           876 non-null    category
dtypes: category(2), int64(5), object(2)
memory usage: 51.2+ KB


In [359]:
# The following data is to be label encoded
dataFrame = pd.get_dummies(dataFrame,columns=["Sex","Embarked","AGE","FARE"])

In [360]:
get_data_info(dataFrame)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 26 columns):
 #   Column         Non-Null Count  Dtype
---  ------         --------------  -----
 0   Survived       891 non-null    int64
 1   Pclass         891 non-null    int64
 2   SibSp          891 non-null    int64
 3   Parch          891 non-null    int64
 4   total_members  891 non-null    int64
 5   Sex_female     891 non-null    uint8
 6   Sex_male       891 non-null    uint8
 7   Embarked_C     891 non-null    uint8
 8   Embarked_Q     891 non-null    uint8
 9   Embarked_S     891 non-null    uint8
 10  AGE_Young      891 non-null    uint8
 11  AGE_Student    891 non-null    uint8
 12  AGE_Teenage    891 non-null    uint8
 13  AGE_Adult      891 non-null    uint8
 14  AGE_Senior     891 non-null    uint8
 15  AGE_Old        891 non-null    uint8
 16  FARE_F1        891 non-null    uint8
 17  FARE_F2        891 non-null    uint8
 18  FARE_F3        891 non-null    uint8
 19  FARE_F4 

In [361]:
dataFrame.to_csv("clean_train.csv")

In [362]:
X = dataFrame.drop("Survived",axis=1)
y = dataFrame["Survived"]

In [363]:
from sklearn.ensemble import GradientBoostingClassifier

from sklearn.model_selection import GridSearchCV

parameters = [{"n_estimators":[100,200,300,400]},
             {"learning_rate":[0.1,0.3,0.5,0.7,0.9,1.1,1.3,1.5,1.7,2.0]},
             {"max_depth":[1,2,3,4,5,6,7,8,9,10]},
              {'max_features': [0.3, 0.2,0.1]},
              {'max_depth': [4, 8]}]
gscv = GridSearchCV(GradientBoostingClassifier(),parameters,verbose=1,n_jobs=-1,cv=100)
gscv.fit(X,y)

Fitting 100 folds for each of 29 candidates, totalling 2900 fits


GridSearchCV(cv=100, estimator=GradientBoostingClassifier(), n_jobs=-1,
             param_grid=[{'n_estimators': [100, 200, 300, 400]},
                         {'learning_rate': [0.1, 0.3, 0.5, 0.7, 0.9, 1.1, 1.3,
                                            1.5, 1.7, 2.0]},
                         {'max_depth': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]},
                         {'max_features': [0.3, 0.2, 0.1]},
                         {'max_depth': [4, 8]}],
             verbose=1)

In [364]:
print(gscv.best_score_)
print(gscv.best_params_)

0.82875
{'max_features': 0.1}


In [365]:
model = GradientBoostingClassifier(max_features=0.1)

In [366]:
model.fit(X,y)

GradientBoostingClassifier(max_features=0.1)

In [367]:
import pickle
pickle.dump(model,open("model1", 'wb'))

In [371]:
dataFrame = pd.read_csv("./test.csv")

In [372]:
dataFrame.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  418 non-null    int64  
 1   Pclass       418 non-null    int64  
 2   Name         418 non-null    object 
 3   Sex          418 non-null    object 
 4   Age          332 non-null    float64
 5   SibSp        418 non-null    int64  
 6   Parch        418 non-null    int64  
 7   Ticket       418 non-null    object 
 8   Fare         417 non-null    float64
 9   Cabin        91 non-null     object 
 10  Embarked     418 non-null    object 
dtypes: float64(2), int64(4), object(5)
memory usage: 36.0+ KB


In [373]:
# Dropping Cabin Column
dataFrame = dataFrame.drop("Cabin",axis=1)

In [374]:
dataFrame = combine_parch_sibsp(dataFrame,dataFrame.Parch,dataFrame.SibSp)

In [375]:
dataFrame = age_to_category(dataFrame,dataFrame.Age)

In [376]:
dataFrame = fare_to_category(dataFrame,dataFrame.Fare)

In [377]:
# The following data is to be label encoded
dataFrame = pd.get_dummies(dataFrame,columns=["Sex","Embarked","AGE","FARE"])

In [378]:
get_data_info(dataFrame)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 28 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   PassengerId    418 non-null    int64 
 1   Pclass         418 non-null    int64 
 2   Name           418 non-null    object
 3   SibSp          418 non-null    int64 
 4   Parch          418 non-null    int64 
 5   Ticket         418 non-null    object
 6   total_members  418 non-null    int64 
 7   Sex_female     418 non-null    uint8 
 8   Sex_male       418 non-null    uint8 
 9   Embarked_C     418 non-null    uint8 
 10  Embarked_Q     418 non-null    uint8 
 11  Embarked_S     418 non-null    uint8 
 12  AGE_Young      418 non-null    uint8 
 13  AGE_Student    418 non-null    uint8 
 14  AGE_Teenage    418 non-null    uint8 
 15  AGE_Adult      418 non-null    uint8 
 16  AGE_Senior     418 non-null    uint8 
 17  AGE_Old        418 non-null    uint8 
 18  FARE_F1        418 non-null   

In [379]:
model = pickle.load(open("./model1" , 'rb'))

In [384]:
predict_params = dataFrame.drop(["PassengerId","Ticket","Name"],axis=1)

In [385]:
get_data_info(predict_params)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 25 columns):
 #   Column         Non-Null Count  Dtype
---  ------         --------------  -----
 0   Pclass         418 non-null    int64
 1   SibSp          418 non-null    int64
 2   Parch          418 non-null    int64
 3   total_members  418 non-null    int64
 4   Sex_female     418 non-null    uint8
 5   Sex_male       418 non-null    uint8
 6   Embarked_C     418 non-null    uint8
 7   Embarked_Q     418 non-null    uint8
 8   Embarked_S     418 non-null    uint8
 9   AGE_Young      418 non-null    uint8
 10  AGE_Student    418 non-null    uint8
 11  AGE_Teenage    418 non-null    uint8
 12  AGE_Adult      418 non-null    uint8
 13  AGE_Senior     418 non-null    uint8
 14  AGE_Old        418 non-null    uint8
 15  FARE_F1        418 non-null    uint8
 16  FARE_F2        418 non-null    uint8
 17  FARE_F3        418 non-null    uint8
 18  FARE_F4        418 non-null    uint8
 19  FARE_F5 

In [386]:
predicted_values = model.predict(predict_params)

In [387]:
print(predicted_values)

[0 0 0 0 1 0 1 0 1 0 0 0 1 0 1 1 0 0 1 1 0 1 1 1 1 0 1 0 0 0 0 0 1 1 1 0 1
 1 0 0 0 0 0 1 1 0 0 0 1 1 0 0 1 1 0 0 0 0 0 1 0 0 0 1 1 1 1 0 0 1 1 0 1 0
 1 0 0 1 0 1 1 0 0 0 0 0 1 1 1 1 1 0 1 0 0 0 1 0 1 0 1 0 0 0 1 0 0 0 0 0 0
 1 1 1 1 0 0 1 1 1 1 0 1 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0 0 0 0 0 0
 0 0 1 0 0 1 0 0 1 1 0 1 1 1 1 0 0 1 0 0 1 1 0 0 0 0 0 1 1 0 1 1 0 1 1 0 1
 0 1 0 0 0 0 0 1 0 1 0 1 1 0 1 1 1 0 1 0 0 1 0 1 0 0 0 0 1 0 0 1 0 1 0 1 0
 1 0 1 1 0 1 0 0 0 1 0 0 1 0 0 0 1 1 1 1 0 0 0 0 1 0 1 1 1 0 1 0 0 0 0 0 1
 0 0 0 1 1 0 0 0 0 1 0 0 0 1 1 0 1 0 0 0 0 1 1 1 1 1 0 0 0 0 0 0 1 0 0 0 0
 1 0 0 0 0 0 0 0 1 1 0 1 0 0 0 0 0 1 1 1 0 0 0 0 0 0 0 0 1 0 1 0 0 0 1 0 0
 1 0 0 0 0 0 0 0 0 0 1 0 1 0 1 0 1 1 0 0 0 1 0 1 0 0 1 0 1 1 0 1 1 0 1 1 0
 0 1 0 0 1 1 1 0 0 0 0 0 1 1 0 1 0 0 0 0 0 1 1 0 0 1 0 1 0 0 1 0 1 0 0 0 0
 0 1 1 1 1 1 0 1 0 0 0]


In [389]:
survived_data = pd.DataFrame(predicted_values,columns=["Survived"])
passender_id = pd.DataFrame(dataFrame["PassengerId"],columns=["PassengerId"]) 
passender_id["Survived"] = survived_data
passender_id.to_csv("submission_file_2.csv",index=False)